In [7]:
library(tseries)
library(vars)


In [2]:
data <- read.csv("./stationary_data.csv")
data

var1,var2,var3
<dbl>,<dbl>,<dbl>
8.778324,0.4549183,3.14660311
4.060246,5.1906868,3.67456524
-5.513257,1.9018173,-0.77639107
-4.536916,-3.6050020,-1.11632610
-3.135693,-2.4210588,-1.37984115
5.493270,-1.5790918,1.00799378
2.897723,3.5143362,0.23645938
2.583402,1.4308751,0.60122078
1.070094,1.3454019,0.18339480


In [3]:
adf_var1 <- adf.test(data$var1, alternative = "stationary")
adf_var2 <- adf.test(data$var2, alternative = "stationary")
adf_var3 <- adf.test(data$var3, alternative = "stationary")

adf_var1
adf_var2
adf_var3


	Augmented Dickey-Fuller Test

data:  data$var1
Dickey-Fuller = -3.6816, Lag order = 2, p-value = 0.04417
alternative hypothesis: stationary



	Augmented Dickey-Fuller Test

data:  data$var2
Dickey-Fuller = -3.8711, Lag order = 2, p-value = 0.03064
alternative hypothesis: stationary



	Augmented Dickey-Fuller Test

data:  data$var3
Dickey-Fuller = -4.0902, Lag order = 2, p-value = 0.02011
alternative hypothesis: stationary


In [ ]:
var_data <- data[, c("var1", "var2", "var3")]

In [9]:
lag_selection <- VARselect(var_data, lag.max = 10, type = "const")
print(lag_selection)
optimal_lag <- lag_selection$selection["AIC(n)"]  

$selection
AIC(n)  HQ(n)  SC(n) FPE(n) 
     1      1      1      1 

$criteria
          1    2    3    4    5    6    7    8    9   10
AIC(n) -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
HQ(n)  -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
SC(n)  -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf -Inf
FPE(n)    0    0    0    0    0    0    0    0    0    0



In [10]:
var_model <- VAR(var_data, p = optimal_lag, type = "const")
summary(var_model)



VAR Estimation Results:
Endogenous variables: var1, var2, var3 
Deterministic variables: const 
Sample size: 11 
Log Likelihood: -21.036 
Roots of the characteristic polynomial:
0.6107 0.6107 0.02007
Call:
VAR(y = var_data, p = optimal_lag, type = "const")


Estimation results for equation var1: 
var1 = var1.l1 + var2.l1 + var3.l1 + const 

        Estimate Std. Error t value Pr(>|t|)
var1.l1   0.7242     0.5977   1.212    0.265
var2.l1  -0.4322     0.6445  -0.670    0.524
var3.l1  -1.6227     1.8684  -0.868    0.414
const     1.4450     1.5061   0.959    0.369


Residual standard error: 4.514 on 7 degrees of freedom
Multiple R-Squared: 0.2987,	Adjusted R-squared: -0.001786 
F-statistic: 0.9941 on 3 and 7 DF,  p-value: 0.4494 


Estimation results for equation var2: 
var2 = var1.l1 + var2.l1 + var3.l1 + const 

         Estimate Std. Error t value Pr(>|t|)    
var1.l1  0.611402   0.007093  86.198 7.45e-12 ***
var2.l1 -0.099487   0.007649 -13.006 3.70e-06 ***
var3.l1 -0.026990   0.0221